In [1]:
#Load libraries
import pandas as pd
import nltk
import matplotlib
import numpy

In [2]:
df = pd.read_csv("small_df.csv",delimiter=',')
df.drop(index=[0], inplace=True)

In [3]:
all_doc = df.to_dict(orient='list')

In [4]:
data = {}
for key in df:
    data[key] = list(df[key])

In [5]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

C:\Users\Master\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_name = "textattack/roberta-base-SST-2" # This code snippet loads the textattack/roberta-base-SST-2 model, which is a RoBERTa model fine-tuned for the SST-2 (Stanford Sentiment Treebank) dataset.
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def Pos_Neg(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    sentiment_class = torch.argmax(logits).item()
    return sentiment_class

In [22]:
data["Subway"][2][4:]

"'This is the nearest Subway to Section 2 Guangfu Road. This has all the latest Sub way items available in Taiwan. You can dine in/ take out.']"

In [12]:
data_summ = {} #take only positive comments for summary
stopword = ["\\","[","]","\r","\n","'",'"',"/"]
for key in data:
    buffer = []
    i=0
    for comm in data[key]:
        if(i):
            comm = comm[4:]
        else:
            i=1
        if(Pos_Neg(comm) == 1):
            comm = [word for word  in comm if word not in stopword]
            comm = "".join(comm)
            buffer.append(comm)
            
    data_summ[key] = " ".join(buffer)

In [14]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [15]:
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [44]:
def Summary(data):
    # Tokenize the input text and convert it to a PyTorch tensor
    inputs = tokenizer(data, return_tensors="pt", max_length=1024, truncation=True)
    # Perform text summarization using the BART model
    with torch.no_grad():
        outputs = model.generate(**inputs)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Display the result
    return summary

In [47]:
summary_dict = {}
for key in data_summ:
        summary_dict[key] = Summary(data_summ[key])


In [2]:
s = ["\xa0","0xa4"]
summary = {}
for key in summary_dict:
    val = summary_dict[key]
    val = [word for word  in val if word not in s]
    val = "".join(val)
    summary[key] = val

NameError: name 'summary_dict' is not defined

In [59]:
import csv

with open('Summaries_of_restaurants.csv', 'w', newline='') as csvfile:
  # 定義欄位
  fieldnames = ['Restaurant', 'Summary']

  # 將 dictionary 寫入 CSV 檔
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

  # 寫入第一列的欄位名稱
  writer.writeheader()

  # 寫入資料
  for key in summary:
    writer.writerow({'Restaurant' : key, 'Summary':summary[key]})